In [12]:
import numpy as np
import pandas as pd
from numpy.linalg import eig
from scipy.linalg import hilbert
from copy import copy

In [98]:
def pow_method(a,eps,x0=None): #степенной метод
    if x0 is None:
        x0 = np.random.uniform(-1,1,size=a.shape[1])
    x1 = a@x0
    num_of_iters = 1
    lambda0 = x1[0]/x0[0]
    while True:
        x0,x1 = x1, a@x1
        lambda1 = x1[0]/x0[0]
        if abs(lambda1-lambda0)<eps or num_of_iters > 5000:
            break
        lambda0 = lambda1
        num_of_iters += 1
    return abs(lambda1),num_of_iters

In [99]:
def scal_method(a,eps,x0=None): #метод скалярных произведений
    if x0 is None:
            x0 = np.random.uniform(-1,1,size=a.shape[1])
    num_of_iters = 1
    x1 = a@x0
    y0 = copy(x0)
    a_T = np.transpose(a)
    y1 = a_T@x0
    lambda0 = np.dot(x1,y1)/np.dot(x0,y0)
    while True:
        x0,x1 = x1, a@x1
        y0,y1 = y1, a_T@y1
        lambda1 = np.dot(x1,y1)/np.dot(x0,y1)
        if abs(lambda1-lambda0)<eps or num_of_iters > 5000:
            break
        lambda0 = lambda1
        num_of_iters += 1
    return abs(lambda1),num_of_iters

Матрицы из учебника Д.К. Фадддева и В.Н. Фаддеевой:

In [100]:
X0 = np.array([[-5.509882,1.870086,0.422908],
              [0.287865,-11.811654,5.7119],
              [0.049099,4.308033,-12.970687]])
X1 = np.array([[4.2,-3.4,0.3],
              [4.7,-3.9,0.3],
              [-5.6,5.2,0.1]])

In [101]:
matrixes = [X0,X1,hilbert(3),hilbert(4)]

In [107]:
X = pd.DataFrame(columns=['eps=10^(-2),res','eps=10^(-2),iters',
                          'eps=10^(-3),res', 'eps=10^(-3),iters',
                          'eps=10^(-4),res','eps=10^(-4),iters',
                         'eps=10^(-5),res','eps=10^(-5),iters'])

Y = pd.DataFrame(columns=['eps=10^(-2),res','eps=10^(-2),iters',
                          'eps=10^(-3),res', 'eps=10^(-3),iters',
                          'eps=10^(-4),res','eps=10^(-4),iters',
                         'eps=10^(-5),res','eps=10^(-5),iters'])

for matrix in matrixes:
    x0 = np.ones(matrix.shape[1])
    lambda_true = max(abs(eig(matrix)[0]))
    row_X,row_Y = [],[]
    for i in range(2,6):
        lambda_pow,pow_iters = pow_method(matrix,10**(-i),x0)
        lambda_scal,scal_iters = scal_method(matrix,10**(-i),x0)
        row_X.extend([abs(lambda_pow-lambda_true),pow_iters])
        row_Y.extend([abs(lambda_scal-lambda_true),scal_iters])
    X = X.append(pd.Series(row_X,index=X.columns),True)
    Y = Y.append(pd.Series(row_Y,index=Y.columns),True)
    
X.index = ['X0','X1','hilbert(3)','hilbert(4)']
Y.index = ['X0','X1','hilbert(3)','hilbert(4)']

In [108]:
X #результаты для степенного метола

,"eps=10^(-2),res","eps=10^(-2),iters","eps=10^(-3),res","eps=10^(-3),iters","eps=10^(-4),res","eps=10^(-4),iters","eps=10^(-5),res","eps=10^(-5),iters"
X0,0.005242,14.0,0.000440,17.0,0.000037,20.0,7.011850e-06,22.0
X1,0.021915,7.0,0.003110,15.0,0.000325,25.0,3.478194e-05,35.0
hilbert(3),0.000218,3.0,0.000019,4.0,0.000002,5.0,1.430000e-07,6.0
hilbert(4),0.000626,3.0,0.000071,4.0,0.000008,5.0,8.962432e-07,6.0


In [109]:
Y #результаты для метода скалярных произведений

,"eps=10^(-2),res","eps=10^(-2),iters","eps=10^(-3),res","eps=10^(-3),iters","eps=10^(-4),res","eps=10^(-4),iters","eps=10^(-5),res","eps=10^(-5),iters"
X0,2.132385e-03,7.0,7.636061e-05,9.0,1.451761e-05,10.0,5.262480e-07,12.0
X1,1.361535e-02,3.0,1.303516e-03,8.0,1.383541e-04,13.0,1.483734e-05,18.0
hilbert(3),9.728686e-07,2.0,9.728686e-07,2.0,7.340026e-09,3.0,7.340026e-09,3.0
hilbert(4),4.832066e-06,2.0,4.832066e-06,2.0,6.142240e-08,3.0,6.142240e-08,3.0
